## 安装 condoa
```bash
 conda create -n ai-learn python=3.12
 conda activate ai-learn
```

In [ ]:
# 执行以下命令安装必要的包
%pip install -U langchain=="0.3.23"
%pip install -U langchain-openai=="0.3.4"

In [ ]:
# 验证langchain版本
import langchain 
print(langchain.__version__)
assert langchain.__version__ == "0.3.23"

0.3.23


In [ ]:
# langchain chat demo
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

chat_model = ChatOpenAI(model="openai/gpt-4o",
    base_url="https://openrouter.ai/api/v1", api_key="adb215b82fc822a595306ea18fce2df4536a1c6325b55c47f538f03eb9441b13")

messages=[
    SystemMessage("Translate the following from English into Chinese."),
    HumanMessage("hi!"),
]


for chunk in chat_model.stream( input= messages ):
    print(chunk.content,end="")






嗨！

In [ ]:
# langgraph & mcp 
#%pip install -qU langchain-mcp-adapters==0.0.9
#%pip install -qU langgraph==0.3.31

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.prebuilt import create_react_agent
import os

load_dotenv()

llm = ChatOpenAI(model="openai/gpt-4o",
    base_url="https://openrouter.ai/api/v1", api_key="fc822a595306ea18fce2df4536a1c6325b55c47f538f03eb9441b13")

async with MultiServerMCPClient(
    {
        "weather": {  # 启动weather服务
            # make sure you start your weather server on port 8000
            "url": "http://localhost:8000/sse",
            "transport": "sse",
        },
        "mcp-server-time": {
            "command": "python",
            "args": ["-m", "mcp_server_time", "--local-timezone=Asia/Shanghai"]
        }
    }
) as client:
    print(".....loading tools.....")
    agent = create_react_agent(llm, client.get_tools())
    weather_response = await agent.ainvoke({"messages": "现在这个时间,北京的天气怎么样?"})
    
    print(weather_response["messages"][-1].content)
    # inputs = {"messages": [("user", "北京的天气怎么样?")]}

    # for chunk in agent.stream( inputs , stream_mode="values"):
    #      message = chunk["messages"][-1]
    #      message.pretty_print()


.....loading tools.....
现在是北京时间 2025年4月21日，晚上7:56。北京的天气晴朗，最高温度为72°F（约22°C）。
